In [1]:
import pandas as pd

In [1]:
def get_gdp_data():
    """Grab GDP data from a CSV file.

    This uses caching to avoid having to read the file every time. If we were
    reading from an HTTP endpoint instead of a file, it's a good idea to set
    a maximum age to the cache with the TTL argument: @st.cache_data(ttl='1d')
    """

    # Instead of a CSV on disk, you could read from an HTTP endpoint here too.
    DATA_FILENAME = r'C:\Users\mikej\Desktop\strokes_from_winner\gdp_data.csv'
    raw_gdp_df = pd.read_csv(DATA_FILENAME)

    MIN_YEAR = 1960
    MAX_YEAR = 2022

    # The data above has columns like:
    # - Country Name
    # - Country Code
    # - [Stuff I don't care about]
    # - GDP for 1960
    # - GDP for 1961
    # - GDP for 1962
    # - ...
    # - GDP for 2022
    #
    # ...but I want this instead:
    # - Country Name
    # - Country Code
    # - Year
    # - GDP
    #
    # So let's pivot all those year-columns into two: Year and GDP
    gdp_df = raw_gdp_df.melt(
        ['Country Code'],
        [str(x) for x in range(MIN_YEAR, MAX_YEAR + 1)],
        'Year',
        'GDP',
    )

    # Convert years from string to integers
    gdp_df['Year'] = pd.to_numeric(gdp_df['Year'])

    return gdp_df

gdp_df = get_gdp_data()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
def get_sales():
    """Grab GDP data from a CSV file.

    This uses caching to avoid having to read the file every time. If we were
    reading from an HTTP endpoint instead of a file, it's a good idea to set
    a maximum age to the cache with the TTL argument: @st.cache_data(ttl='1d')
    """

    # Instead of a CSV on disk, you could read from an HTTP endpoint here too.
    # DATA_FILENAME = r"C:\Users\mikej\OneDrive - Wilde Brands\data\outputs\true_sales_cust.csv"
    DATA_FILENAME = r'true_sales_cust.csv'
    sales = pd.read_csv(DATA_FILENAME)

    sales['date'] = pd.to_datetime(sales['date'])

    sales = sales[sales.status == 'closed']

    sales = sales.groupby(
        [pd.Grouper(key='date',freq='W'), 'cust_parent_name']).agg({'amount':'sum'}).reset_index().astype({'amount': 'int64','cust_parent_name':'string'})

    return sales

sales = get_sales()

In [39]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   date              1459 non-null   datetime64[ns]
 1   cust_parent_name  1459 non-null   string        
 2   amount            1459 non-null   int64         
dtypes: datetime64[ns](1), int64(1), string(1)
memory usage: 34.3 KB


In [40]:
# get top 10 customers by sales amount

sales[sales.date.dt.year > 2024].groupby('cust_parent_name').agg({'amount':'sum'}).sort_values('amount',ascending=False).head(10).index.tolist()

['Costco',
 'Kehe',
 'UNFI',
 'Amazon',
 'Target',
 'Walmart',
 'Grocery Outlet',
 'Set Sail',
 'Kroger',
 'HEB']

In [41]:
sales.head(3)

,date,cust_parent_name,amount
0,2024-01-07,Amazon,37028
1,2024-01-07,Capitol Distributing,4637
2,2024-01-07,Core-Mark,1917


In [42]:
# SLIDER VALUES
min_value = sales.date.min()
max_value = sales.date.max()

print(min_value)
print(max_value)

2024-01-07 00:00:00
2025-03-02 00:00:00


In [43]:
# get unique parent customers
parent_customers = sorted(sales['cust_parent_name'].unique())
# parent_customers

In [44]:
# get top 10 parent_customers by sales
parent_customers_to_show = sales[
    sales.date.dt.year > 2024
    ].groupby(
        'cust_parent_name'
        ).agg({'amount':'sum'}
              ).sort_values('amount',ascending=False
                            ).head(10).index.tolist()
parent_customers_to_show

['Costco',
 'Kehe',
 'UNFI',
 'Amazon',
 'Target',
 'Walmart',
 'Grocery Outlet',
 'Set Sail',
 'Kroger',
 'HEB']

In [48]:
# Filter the data

to_date = max_value
from_date = min_value

selected_parent_customers = ['UNFI','Kehe']

filtered_sales = sales[
    (sales['cust_parent_name'].isin(selected_parent_customers))
    & (sales['date'] <= to_date)
    & (from_date <= sales['date'])
]

filtered_sales.head(3)

,date,cust_parent_name,amount
8,2024-01-07,Kehe,139990
16,2024-01-07,UNFI,3757
31,2024-01-14,Kehe,135062


In [49]:
# line chart

line_chart = px.line(
    filtered_sales,
    x='date',
    y='amount',
    color='cust_parent_name',
).update_layout(showlegend=False)

line_chart.show()

<IPython.core.display.Javascript object>

In [50]:
# YoY Camparisions

first_date = sales[sales['date'] == from_date]
last_date = sales[sales['date'] == to_date]

print(first_date.head(3))
print(last_date.head(3))

        date      cust_parent_name  amount
0 2024-01-07                Amazon   37028
1 2024-01-07  Capitol Distributing    4637
2 2024-01-07             Core-Mark    1917
           date cust_parent_name  amount
1438 2025-03-02           Amazon   53900
1439 2025-03-02              CVS    1346
1440 2025-03-02        Core-Mark    9846


In [62]:
import math

for i, parent_customer in enumerate(selected_parent_customers):
    first_sales = first_date[first_date['cust_parent_name'] == parent_customer]['amount'].sum()
    last_sales = last_date[last_date['cust_parent_name'] == parent_customer]['amount'].sum()
    print(f'{parent_customer} first sales: {first_sales} last sales: {last_sales} YoY: {last_sales-first_sales}')

for i, parent_customer in enumerate(selected_parent_customers):
        first_parent_cust = first_date[sales['cust_parent_name'] == parent_customer]['amount'].sum() / 1_00
        last_parent_cust = last_date[sales['cust_parent_name'] == parent_customer]['amount'].sum() / 1_00

        if math.isnan(first_parent_cust):
            growth = 'n/a'
            delta_color = 'off'
        else:
            growth = f'{last_parent_cust - first_parent_cust:,.2f}x'
            delta_color = 'normal'

            print('===')
            print(' ')
            print(f'Parent Customer {parent_customer}')
            print(' ')
            print(f'Last Parent Cust: {last_parent_cust:,.2f}') 
            print(' ')
            print(f'Growth: {growth}')
            print(f'Delta Color{delta_color}')
            print('---------')


UNFI first sales: 3757 last sales: 98322 YoY: 94565
Kehe first sales: 139990 last sales: 0 YoY: -139990
===
 
Parent Customer UNFI
 
Last Parent Cust: 983.22
 
Growth: 945.65x
Delta Colornormal
---------
===
 
Parent Customer Kehe
 
Last Parent Cust: 0.00
 
Growth: -1,399.90x
Delta Colornormal
---------


C:\Users\mikej\AppData\Local\Temp\ipykernel_20116\875602102.py:9: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

C:\Users\mikej\AppData\Local\Temp\ipykernel_20116\875602102.py:10: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

